In [79]:
import os
from datetime import datetime
import csv
from gym import spaces
import numpy as np
from scipy.optimize import fsolve


seed = 0
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)


class csv_handler:
    def __init__(self, filename, fieldnames):
        self.fieldnames = fieldnames
        file_exists = os.path.isfile(filename) 
        self.f = open(filename, 'a', newline='')
        self.csv_writer = csv.DictWriter(self.f, fieldnames=fieldnames, delimiter=',')
        if not file_exists:
            self.csv_writer.writeheader()

    def addLine(self, in_dict):
        if len(in_dict.keys()) != len(self.fieldnames):
            print(in_dict.keys())
            print(in_dict)
            print(self.fieldnames)
        
        assert len(in_dict.keys()) == len(self.fieldnames)
        self.csv_writer.writerow(in_dict)
        self.f.flush()

    def close_csv(self):
        self.f.close()



class EnvTank:
    def __init__(self, algo, noise, create_log=1, timestamp='', save_contexts=0, contexts_file='', ver=1):
        
        assert noise is not None
        self.noise_dist = noise['dist']
        self.noise_var = noise['var']
        self.create_log = create_log
        self.save_contexts = save_contexts
        self.algo = algo
        self.version = ver
        
        ### START KINEMATICS
        self.vel = 1000
        self.grav = -9.81

        self.elmin = np.deg2rad(0)
        velrgdistmin = self.vel * np.cos(self.elmin)
        velhtdistmin = self.vel * np.sin(self.elmin)
        t_distmin = np.roots([0.5 * self.grav, velhtdistmin, 0])[0]
        self.distmin = velrgdistmin * t_distmin

        self.elopt = np.deg2rad(45)
        velrgopt = self.vel * np.cos(self.elopt)
        velhtopt = self.vel * np.sin(self.elopt)
        self.t_distmax = np.roots([0.5 * self.grav, velhtopt, 0])[0]
        self.distmax = velrgopt * self.t_distmax

        self.elmax = np.deg2rad(90)
        velhtmax = self.vel * np.sin(self.elmax)
        self.t_max = np.roots([0.5 * self.grav, velhtmax, 0])[0]
        ### END KINEMATICS
        
        self.step_counter = 0
        min_action, max_action, action_dim = 0, 2 * self.elmax, 1
        # min_action, max_action, action_dim = 0, self.elmax, 1
        min_obs, max_obs = -0.5 * self.distmax, 0.5 * self.distmax
        self.obs_dim = 2 if self.version == 2 else 1
        
        self.action_space = spaces.Box(low=min_action, high=max_action, shape=(action_dim,), dtype=np.float32)
        self.observation_space = spaces.Box(low=min_obs, high=max_obs, shape=(self.obs_dim,), dtype=np.float32)
        
        # self.obs = self.observation_space.sample()
        self.get_obs()
                
        self.timestamp = datetime.now().strftime("%Y%m%d%H%M%S") if timestamp=='' else timestamp
        self.path = os.path.join(os.getcwd(), 'results', self.timestamp)
        
        if self.save_contexts:
            assert len(contexts_file) == 0
            fieldnames = ['step', 'context0', 'context1']
            filename = os.path.join(self.path, self.timestamp+'_context_log.csv')
            os.makedirs(self.path, exist_ok=True)
            self.csv_h_contexts = csv_handler(filename, fieldnames)
            
            # context2 = self.obs[2] if self.version == 2 else -1
            measure = {'step' : self.step_counter, 'context0' : self.obs[0], 'context1': self.obs[1]}
            self.csv_h_contexts.addLine(measure)
            
        self.reset(contexts_file)


    def get_obs(self):
        self.obs = self.observation_space.sample()
        obs0 = self.obs[0]
        obs1 = self.obs[1]
        if obs0 > obs1:
            self.obs[0] = obs1
            self.obs[1] = obs0


    def reset(self, contexts_file=''):
        
        if self.create_log:
            fieldnames = ['step', 'context', 'action', 'reward', 'constraint_1', 'alpha']
            filename = os.path.join(self.path, self.timestamp+f'_{self.algo}_log.csv')
            os.makedirs(self.path, exist_ok=True)
            self.csv_h = csv_handler(filename, fieldnames)
        
        
        if len(contexts_file) > 0:
            with open(contexts_file) as in_file:
                self.n_lines = sum(1 for _ in in_file)
            
            self.f = open(contexts_file)
            self.contexts_file = csv.reader(self.f, delimiter=',')
            next(self.contexts_file)
            line = next(self.contexts_file)
            self.obs = np.zeros(self.obs_dim)
            
            for i in range(self.obs_dim):
                self.obs[i] = float(line[i+1])
        else:
            self.contexts_file = None
    
    def close(self):
        if self.create_log:
            self.csv_h.close_csv()
        if self.save_contexts:
            self.csv_h_contexts.close_csv()
        if self.contexts_file is not None:
            self.f.close()


    def compute_reward(self, action):
        # print('ENV_TANK action[0]:\n{}'.format(action[0]))
        el = action[0]
        # print('\nCOMPUTE_REWARD el:\n{}\nobs:\n{}\n'.format(el, obs))
        velht = self.vel * np.sin(el)
        veldist = self.vel * np.cos(el)
        self.t_impact = np.roots([0.5 * self.grav, velht, 0])[0]

        self.dist_impact = veldist * self.t_impact
        self.loc_impact = self.obs[0] + np.array(self.dist_impact)
        
        self.hit_miss_dist = np.subtract(self.obs[1], self.loc_impact)
        dreward = np.abs(self.hit_miss_dist) / np.abs(self.obs[1] - self.obs[0])
        # print('DIAG hit_miss_dist: {}, diff: {}, dreward: {}'.format(self.hit_miss_dist, self.obs[1] - self.obs[0], dreward))
        # dreward = np.abs(self.hit_miss_dist)
        treward = self.t_impact

        # if self.hit_miss_dist < (self.dist * self.hitthresh):
        #     reward = rewardbase + self.rewardmax
        # else:
        #     reward = rewardbase

        return treward, dreward
        # return dreward, treward


    def compute_truthscore(self):  # accessing from outside
        def t_hit_opt(x):
            z = x - self.dist_impact /\
                (self.vel * np.cos(np.arcsin(-0.5 * self.grav * x / self.vel)))

            return z

        t_opt = fsolve(t_hit_opt, self.t_distmax / 2)[0]
        tdiff = np.abs(self.t_impact - t_opt)
        truthscore = np.array([self.hit_miss_dist, tdiff])

        return truthscore, t_opt


    def step(self, action, alpha=None):

        m1, m2 = self.compute_reward(action)
        truthscore, t_opt = self.compute_truthscore()
        
        if self.noise_dist == 'norm':
            # m1 += np.random.normal(loc=0.0, scale=self.noise_var)
            # m2 += np.random.normal(loc=0.0, scale=self.noise_var)
            pert_m1 = np.random.normal(loc=0.0, scale=self.noise_var[0])
            if m1 + pert_m1 > 0:
                m1 -= pert_m1
            elif m1 + pert_m1 <= 0:
                m1 += pert_m1
            if self.version == 2:
                # m3 += np.random.normal(loc=0.0, scale=self.noise_var)
                pert_m2 = np.random.normal(loc=0.0, scale=self.noise_var[1])
                # print(pert_m1)
                if m2 + pert_m2 > 0:
                    m2 -= pert_m2
                elif m2 + pert_m2 <= 0:
                    m2 += pert_m2
            
        if self.create_log:
            measure = {'step' : self.step_counter, 'context' : self.obs, 'action': action, 'reward' : m1, 'constraint_1': m2, 'alpha': alpha}
            self.csv_h.addLine(measure)
            
        if self.contexts_file is not None:
            if self.step_counter >= self.n_lines:
                raise Exception('Context file ended.')
            line = next(self.contexts_file)
            for i in range(self.obs_dim):
                self.obs[i] = float(line[i+1])
        else:
            # self.obs = self.observation_space.sample()
            self.get_obs()
        self.step_counter += 1
        
        if self.save_contexts:
            # context2 = self.obs[2] if self.version == 2 else -1
            measure = {'step' : self.step_counter, 'context0' : self.obs[0], 'context1': self.obs[1]}
            self.csv_h_contexts.addLine(measure)
        
        obs = np.expand_dims(self.obs, axis=0)
        reward = (m1, (m2)) if self.version == 2 else (m1)
        done = False
        info = {}

        return obs, reward, done, info

    

    def set_obs(self, obs):
        self.obs = obs



env_noise = {'dist' : 'norm', 'var' : [1e-1, 1e-4]}
env = EnvTank(algo=None, noise=env_noise, timestamp='', contexts_file='', ver=2)

In [47]:
def compute_scenario(obs0, action):
    el = np.deg2rad(action)
    
    vel = 1000
    grav = -9.81

    velht = vel * np.sin(el)
    veldist = vel * np.cos(el)
    t_impact = np.roots([0.5 * grav, velht, 0])[0]
    # print(t_impact)

    

    dist_impact = veldist * t_impact
    loc_impact = obs0 + np.array(dist_impact)

    return loc_impact


loc_impact = compute_scenario(-39881.48, 24.68140488448044)
37446.914 - loc_impact

# 0.5 * 1019.3679918450559

-26.217120315719512

In [81]:
def test_env():
    def test_loop(validate):
        print('validate {}'.format(validate))
        for scenpair in scenpairs:
            shooterpos = scenpair[0]
            actionideal = scenpair[1]
            targetpos = compute_scenario(shooterpos, actionideal)
            env.set_obs(np.array([shooterpos, targetpos]))
            if validate == 'hits':
                actionactual = actionideal
            elif validate == 'miss_over':
                actionactual = actionideal + 1
            elif validate == 'miss_under':
                actionactual = actionideal - 1
            elif validate == 'miss_big':
                actionactual = actionideal + 10
            elif validate == 'max_err':
                actionactual = 135
            _, reward, _, _ = env.step([np.deg2rad(actionactual)])
            print('shtpos: {:.3f} tgtpos: {:.3f}, reward = {}'.format(shooterpos, targetpos, reward))
        print()
            
    scenpairs = [[0, 22.5], [0, 45], [0, 67.5],
                [0, 112.5], [0, 135], [0, 157.5],
                [-500, 22.5], [-500, 45], [-500, 67.5],
                [500, 112.5], [500, 135], [500, 157.5],] 
    for validate in ['hits', 'miss_over', 'miss_under', 'miss_big', 'max_err']:
        test_loop(validate)


test_env()

validate hits
shtpos: 0.000 tgtpos: 72080.202, reward = (77.84264315787821, -4.001572083672233e-05)
shtpos: 0.000 tgtpos: 101936.799, reward = (144.06253011321988, -0.00022408931992014578)
shtpos: 0.000 tgtpos: 72080.202, reward = (188.16788895354088, -9.77277879876411e-05)
shtpos: 0.000 tgtpos: -72080.202, reward = (188.25963591080333, -1.513572082976979e-05)
shtpos: 0.000 tgtpos: -101936.799, reward = (144.17072579680985, -4.1059850193837235e-05)
shtpos: 0.000 tgtpos: -72080.202, reward = (78.00464403535891, -0.00014542735069629752)
shtpos: -500.000 tgtpos: 71580.202, reward = (77.94294461996029, -1.2167501649282842e-05)
shtpos: -500.000 tgtpos: 101436.799, reward = (144.1160175883559, -3.336743273742668e-05)
shtpos: -500.000 tgtpos: 71580.202, reward = (188.20523684524014, -2.051582637658009e-05)
shtpos: 500.000 tgtpos: -71580.202, reward = (188.3233379823908, -8.540957393017248e-05)
shtpos: 500.000 tgtpos: -101436.799, reward = (144.41570289321388, -6.536185954403606e-05)
shtpos: 5

In [11]:
var = [1e-1,1e-4]
print('var: {}'.format(var))

var: [0.1, 0.0001]


In [27]:
def denormalize_action(action):
    action_low = 0
    action_high = 2 * np.deg2rad(90)
    action = np.minimum(action, 1)
    action = np.maximum(action, 0)
    denorm_act = action * (action_high - action_low) + action_low
    return denorm_act

actions = np.linspace(0, 1, 33)
for action in actions:
    denorm_act = denormalize_action(action)
    print('action: {}, denorm_act: {}'.format(action, denorm_act))

action: 0.0, denorm_act: 0.0
action: 0.03125, denorm_act: 0.09817477042468103
action: 0.0625, denorm_act: 0.19634954084936207
action: 0.09375, denorm_act: 0.2945243112740431
action: 0.125, denorm_act: 0.39269908169872414
action: 0.15625, denorm_act: 0.4908738521234052
action: 0.1875, denorm_act: 0.5890486225480862
action: 0.21875, denorm_act: 0.6872233929727672
action: 0.25, denorm_act: 0.7853981633974483
action: 0.28125, denorm_act: 0.8835729338221293
action: 0.3125, denorm_act: 0.9817477042468103
action: 0.34375, denorm_act: 1.0799224746714913
action: 0.375, denorm_act: 1.1780972450961724
action: 0.40625, denorm_act: 1.2762720155208536
action: 0.4375, denorm_act: 1.3744467859455345
action: 0.46875, denorm_act: 1.4726215563702154
action: 0.5, denorm_act: 1.5707963267948966
action: 0.53125, denorm_act: 1.6689710972195777
action: 0.5625, denorm_act: 1.7671458676442586
action: 0.59375, denorm_act: 1.8653206380689396
action: 0.625, denorm_act: 1.9634954084936207
action: 0.65625, denorm_ac